In [1]:
import os
import re
import uuid
import glob
from typing import List, Dict, Tuple, Optional

import pandas as pd
from dotenv import load_dotenv
from fastapi import FastAPI, HTTPException, Query
from fastapi.responses import RedirectResponse
from pydantic import BaseModel

# OpenAI (Azure)
from openai import AzureOpenAI

In [2]:
# LangChain (modern split)
from langchain_openai import AzureOpenAIEmbeddings
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_community.retrievers import AzureAISearchRetriever
# from langchain_community.vectorstores import AzureSearch
from langchain_text_splitters import TokenTextSplitter

# Azure AI Search vector store
from langchain_azure_ai.vectorstores.azuresearch import AzureSearch

load_dotenv()

True

In [3]:
# ---------- Environment ----------
CSV_PATH = os.getenv(
    "CSV_PATH",
    "/gdpr_cased_articles_with_recitals.csv", 
)

In [4]:
# ---------- Config / Clients ----------

# AOAI_ENDPOINT = os.environ["AZURE_OPENAI_ENDPOINT"]
# AOAI_KEY = os.environ["AZURE_OPENAI_ENDPOINT_API_KEY"]
# CHAT_DEPLOYMENT = os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT"]
# EMB_DEPLOYMENT = os.environ["EMBEDDING_MODEL_ENDPOINT"]
# AOAI_CHAT_API_VERSION = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_API_KEY")
# AOAI_EMB_API_VERSION = os.getenv("EMBEDDING_MODEL_ENDPOINT_API_KEY")
# AOAI_API_VERSION = os.getenv("OPENAI_API_VERSION")

# SEARCH_ENDPOINT = os.environ["VECTOR_DATABASE_ENDPOINT"]  
# SEARCH_KEY = os.environ["VECTOR_DATABASE_ENDPOINT_API_KEY"]
# SEARCH_INDEX = os.getenv("AZURE_SEARCH_INDEX_NAME")

# Remove legacy OpenAI globals that can clash with Azure mode
for k in ("OPENAI_API_BASE", "OPENAI_BASE_URL", "OPENAI_ORG"):
    os.environ.pop(k, None)

In [ ]:
# # ---------- Clients ----------
# chat_client = AzureOpenAI(
#     azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
#     api_key=os.getenv("AZURE_OPENAI_KEY"),
#     api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
# )

In [5]:
embeddings = AzureOpenAIEmbeddings(
    model=os.getenv("AZURE_OPENAI_EMBEDDING_MODEL_DEPLOYMENT_NAME"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    openai_api_key=os.getenv("AZURE_OPENAI_KEY"),
)

In [6]:
# Should return a list of floats of expected length (e.g., 3072 for text-embedding-3-large)
print(len(embeddings.embed_query("hello world")))

3072


In [ ]:
# If you know your embedding model's dimension, set it here (recommended):
# text-embedding-3-large -> 3072; text-embedding-3-small -> 1536
EMBED_DIM = int(os.getenv("AZURE_OPENAI_EMBEDDING_DIM", "3072"))

# Azure Semantic Search
What semantic search adds (if you set semantic_search_enabled=True)

Azure’s Semantic Search is a different, proprietary reranker layer (part of Azure AI Search, not OpenAI).
When you enable it:

- It uses Microsoft’s internal transformer models to re-rank the top n text results.

- It can perform semantic captions, highlights, and reasoning over plain text (not just vectors).

- It’s great for enterprise keyword-based search (“meaningful snippets”, “why did this doc match?”).

**It requires:**

The Semantic Search feature enabled on your Search service (Standard S or higher).

A semantic configuration defined in your index (usually named "default" or "semanticconfig1").

**However:**

The built-in semantic ranker is not aware of your OpenAI embeddings.
- It operates on text fields, not vector fields.

- It costs extra (depending on your tier) and slightly increases latency.

- It’s redundant if your retrieval is already vector-based and you do reranking in LangChain or the LLM itself.

In [7]:
REQUIRED_COLS = {"article_id", "article_title", "article_text", "article_recitals"}

def _normalize_recitals(value) -> list[str]:
    if value is None or (isinstance(value, float) and pd.isna(value)):
        return []
    s = str(value).strip().strip('"').strip("'")
    if not s:
        return []
    return [x.strip() for x in s.split(",") if x.strip()]

def _sort_recitals(rec_set: set[str]) -> list[str]:
    def sort_key(x: str):
        xs = x.strip()
        return (0, int(xs)) if xs.isdigit() else (1, xs.lower())
    return sorted(rec_set, key=sort_key)

def _compose_article_text(title: str, text: str, recitals: list[str]) -> str:
    t = (text or "").strip()
    title = (title or "").strip()
    rec_str = ", ".join(recitals) if recitals else "—"
    return f"## {title}\n\n{t}\n\n[Recitals: {rec_str}]\n"

def load_csv_documents(csv_path: str, merge_by_article_id: bool = True) -> list[Document]:
    if not os.path.isfile(csv_path):
        raise FileNotFoundError(f"CSV not found: {csv_path}")

    df = pd.read_csv(csv_path)  # add encoding=... if needed
    missing = REQUIRED_COLS - set(df.columns)
    if missing:
        raise ValueError(f"CSV missing required columns: {sorted(missing)}")

    # Replace real NaNs only, then cast to str
    for col in ["article_id", "article_title", "article_text", "article_recitals"]:
        df[col] = df[col].where(df[col].notna(), "")
        df[col] = df[col].astype(str)

    if not merge_by_article_id:
        docs: list[Document] = []
        for idx, row in df.iterrows():
            recs = _normalize_recitals(row["article_recitals"])
            content = _compose_article_text(row["article_title"], row["article_text"], recs)
            if content.strip():
                docs.append(Document(
                    page_content=content,
                    metadata={
                        "source": csv_path,
                        "doc_id": str(uuid.uuid4()),
                        "row_index": int(idx),
                        "article_id": row["article_id"].strip(),
                        "article_title": row["article_title"].strip(),
                        "recitals": recs,
                        "merge_mode": "row",
                    }
                ))
        return docs

    # Merge rows with same article_id (title can vary slightly)
    grouped: dict[str, dict[str, object]] = {}
    for idx, row in df.iterrows():
        aid = row["article_id"].strip()
        title = row["article_title"].strip()
        text = (row["article_text"] or "").strip()
        recs = _normalize_recitals(row["article_recitals"])

        if aid not in grouped:
            grouped[aid] = {"titles": [], "texts": [], "recitals": set(), "rows": []}
        if title:
            grouped[aid]["titles"].append(title)
        if text:
            grouped[aid]["texts"].append(text)
        grouped[aid]["recitals"].update(recs)
        grouped[aid]["rows"].append(int(idx))

    docs: list[Document] = []
    for aid, bundle in grouped.items():
        merged_text = "\n\n".join(bundle["texts"]).strip()
        # choose the most common title for this article_id
        titles = [t for t in bundle["titles"] if t]
        title = max(set(titles), key=titles.count) if titles else aid
        rec_list = _sort_recitals(bundle["recitals"])
        content = _compose_article_text(title, merged_text, rec_list)
        if content.strip():
            docs.append(Document(
                page_content=content,
                metadata={
                    "source": csv_path,
                    "doc_id": str(uuid.uuid4()),
                    "article_id": aid,
                    "article_title": title,
                    "row_indices": bundle["rows"],
                    "recitals": rec_list,
                    "merge_mode": "article_id",
                }
            ))
    return docs

In [8]:
docs_generated = load_csv_documents(csv_path=CSV_PATH)

In [9]:
print(docs_generated[2])

page_content='## Territorial scope

This Regulation applies to the processing of personal data in the context of the activities of an establishment of a controller or a processor in the Union, regardless of whether the processing takes place in the Union or not.

This Regulation applies to the processing of personal data of data subjects who are in the Union by a controller or processor not established in the Union, where the processing activities are related to: the offering of goods or services, irrespective of whether a payment of the data subject is required, to such data subjects in the Union;

This Regulation applies to the processing of personal data of data subjects who are in the Union by a controller or processor not established in the Union, where the processing activities are related to: the monitoring of their behaviour as far as their behaviour takes place within the Union.

This Regulation applies to the processing of personal data by a controller not established in the 

In [10]:
len(docs_generated)

99

In [11]:
# ---------- Chunking ----------
def chunk_docs(
    docs: List[Document],
    chunk_size: int = 1000,
    chunk_overlap: int = 0,
) -> List[Document]:
    """
    GDPR articles are long but formal; 1000/200 works well.
    Includes per-chunk offsets & ids for better citations.
    """
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", " ", ""],
    )

    chunks: List[Document] = []
    for d in docs:
        pieces = splitter.split_text(d.page_content)
        for i, text in enumerate(pieces):
            # Estimate character offsets in a simple way
            char_start = sum(len(p) for p in pieces[:i]) - max(i * chunk_overlap, 0)
            char_start = max(char_start, 0)
            char_end = char_start + len(text)
            meta = dict(d.metadata)
            meta.update({
                "chunk_id": i,
                "chunk_count": len(pieces),
                "char_start": char_start,
                "char_end": char_end,
            })
            chunks.append(Document(page_content=text, metadata=meta))
    return chunks

In [12]:
chunks_generated = chunk_docs(docs_generated)

In [13]:
print(chunks_generated[5])

page_content='## Definitions

'personal data' means any information relating to an identified or identifiable natural person ('data subject'); an identifiable natural person is one who can be identified, directly or indirectly, in particular by reference to an identifier such as a name, an identification number, location data, an online identifier or to one or more factors specific to the physical, physiological, genetic, mental, economic, cultural or social identity of that natural person;

'processing' means any operation or set of operations which is performed on personal data or on sets of personal data, whether or not by automated means, such as collection, recording, organisation, structuring, storage, adaptation or alteration, retrieval, consultation, use, disclosure by transmission, dissemination or otherwise making available, alignment or combination, restriction, erasure or destruction;' metadata={'source': '/Users/mukul/Desktop/rag_azure_fastapi/dataset_files/gdpr_cased_arti

In [14]:
len(chunks_generated)

371

In [15]:
# Vector store (Azure AI Search)
# semantic_search_enabled=False
# Use the vector index for retrieval (cosine / dot-product similarity on embeddings) — do not call the Azure semantic ranker service.
def make_vectorstore() -> AzureSearch:
    """
    Pure vector retrieval (fast + cheap). You can enable Azure Semantic Ranker later if needed.
    """
    return AzureSearch(
        azure_search_endpoint=os.getenv("AZURE_SEARCH_ENDPOINT"),
        azure_search_key=os.getenv("AZURE_SEARCH_API_KEY"),
        index_name=os.getenv("AZURE_SEARCH_INDEX_NAME"),
        embedding_function=embeddings.embed_query,
)

In [16]:
vs = make_vectorstore()

In [17]:
vs.add_documents(chunks_generated)

['YzBiN2QzYTEtNzQzNS00NjdmLWFkOTktMGM4NTIxNTRiM2Rh',
 'MjI3MTJlZTItMTMxNC00ODYwLThiZmMtZDcwNjI3YjQ4N2I5',
 'ZDA5MGRkM2YtYzljMC00YzJiLTllZTEtMmQ4MjhjNmU4NDI0',
 'ZjM4ZTBiMDctNzE1Mi00ZTljLTgxYWEtNTNhMTliNmNiZjM0',
 'NjU3NzAyY2MtMGNiZC00MzczLThmY2ItOTU4MWM1NzU1MWYw',
 'ZTRiYmI3OGYtNGY1OS00YWVhLWJhYTctZjNmNWI3MjExNDAz',
 'NTRlOTUzNTktZTUxMS00MGUyLWE4MjUtNWMyNjA4YzJjZTc3',
 'MzlkYTM4NzgtZmQwYS00NTU4LWI4ZjItYzQ1MmEyMmJlOWYy',
 'OGVhMDk0NjMtNmMwZi00ZGE2LTgwNWItMWRjMjM0MTQ4NWQ5',
 'OWZiMDQwNmYtMWZkZS00ZTk0LWE1OGUtMmIyMjNiNmUyYmYw',
 'ODRkZGIwZDEtNzU1ZS00MzU0LThmOTgtZWU0ZjA2NDQ5ZThj',
 'YTRlMTQzMmYtMjFlNC00YWQ2LTk1YjYtMzQ5MTg4NDlkNTY0',
 'MjFjODk0MzgtMDBkMy00YTZiLWI3NTgtNTMzZjVhOGYxZTNi',
 'YWIyZDhhOGQtZDI2MS00MjE3LWI4ODEtMWI1ZWVjNDlhZjE3',
 'ZmJiOTk2YjYtNWUxNS00Y2JhLTk1YmYtODcyMDFkNWYwNDg0',
 'MGY2NzEzYTUtZDhiZi00MjAxLTg1ZWUtNGI1NzhjZWE4ODFk',
 'ZTYxOWE1MmItNWEwMy00NzNiLWFhNWQtNTExYjgzYTYyMjAx',
 'YmFlYzMyYWYtYWE4MC00NjhjLTk0YjktNzU1OThlMmIyMThl',
 'NDBiZjVkYzctNTM4NS00YTY2LWI2YmYtZTQ4NDM0NWI0

Here are 10 retrieval-friendly queries you can run against your GDPR vector store (each targets a specific article/topic present in your CSV):

1. What are the lawful bases for processing under GDPR Article 6?
2. What activities are covered by the GDPR’s territorial scope in Article 3, including non-EU controllers targeting people in the EU?
3. List the core principles for processing personal data in GDPR Article 5.
4. What conditions must be met for valid consent, and how can consent be withdrawn, according to Article 7?
5. When is a Data Protection Impact Assessment required under Article 35, and what does it need to consider?
6. Within how many hours must a controller notify the supervisory authority of a personal data breach under Article 33, and what details must be included?
7. When must a controller communicate a personal data breach to data subjects under Article 34?
8. What information must be provided to data subjects when data are collected directly from them under Article 13?
9. What contractual requirements apply to processors and sub-processors under Article 28?
10. What are the administrative fine tiers and assessment criteria in GDPR Article 83?


In [18]:
docs_queried = vs.similarity_search_with_relevance_scores(
    query="What are the lawful bases for processing under GDPR Article 6?",
    k=5,
)

In [19]:
print(dir(docs_queried[0][0]))

['__abstractmethods__', '__annotations__', '__class__', '__class_getitem__', '__class_vars__', '__copy__', '__deepcopy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__fields__', '__fields_set__', '__format__', '__ge__', '__get_pydantic_core_schema__', '__get_pydantic_json_schema__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__pretty__', '__private_attributes__', '__pydantic_complete__', '__pydantic_computed_fields__', '__pydantic_core_schema__', '__pydantic_custom_init__', '__pydantic_decorators__', '__pydantic_extra__', '__pydantic_fields__', '__pydantic_fields_set__', '__pydantic_generic_metadata__', '__pydantic_init_subclass__', '__pydantic_on_complete__', '__pydantic_parent_namespace__', '__pydantic_post_init__', '__pydantic_private__', '__pydantic_root_model__', '__pydantic_serializer__', '__pydantic_setattr_handlers__', '__pydantic_

In [20]:
print(docs_queried[0][0].page_content)

## Lawfulness of processing

Processing shall be lawful only if and to the extent that at least one of the following applies:  the data subject has given consent to the processing of his or her personal data for one or more specific purposes;

Processing shall be lawful only if and to the extent that at least one of the following applies: processing is necessary for the performance of a contract to which the data subject is party or in order to take steps at the request of the data subject prior to entering into a contract;

Processing shall be lawful only if and to the extent that at least one of the following applies: processing is necessary for compliance with a legal obligation to which the controller is subject;

Processing shall be lawful only if and to the extent that at least one of the following applies: processing is necessary in order to protect the vital interests of the data subject or of another natural person;


In [21]:
print(docs_queried[1][0].page_content)

## Lawfulness of processing

Processing shall be lawful only if and to the extent that at least one of the following applies:  the data subject has given consent to the processing of his or her personal data for one or more specific purposes;

Processing shall be lawful only if and to the extent that at least one of the following applies: processing is necessary for the performance of a contract to which the data subject is party or in order to take steps at the request of the data subject prior to entering into a contract;

Processing shall be lawful only if and to the extent that at least one of the following applies: processing is necessary for compliance with a legal obligation to which the controller is subject;

Processing shall be lawful only if and to the extent that at least one of the following applies: processing is necessary in order to protect the vital interests of the data subject or of another natural person;


In [22]:
print(docs_queried[4][0].page_content)

Processing shall be lawful only if and to the extent that at least one of the following applies: processing is necessary for the performance of a task carried out in the public interest or in the exercise of official authority vested in the controller;

Processing shall be lawful only if and to the extent that at least one of the following applies: processing is necessary for the purposes of the legitimate interests pursued by the controller or by a third party, except where such interests are overridden by the interests or fundamental rights and freedoms of the data subject which require protection of personal data, in particular where the data subject is a child. This shall not apply to processing carried out by public authorities in the performance of their tasks.


In [ ]:
chat_client = AzureOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT"),
    api_key=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
)

In [26]:
deployment = "gpt-4o"
response = chat_client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant that helps GDPR Compliance related topics.",
        },
        {
            "role": "user",
            "content": "What are the lawful bases for processing under GDPR Article 6?",
        },
        {
            "role": "assistant", 
            "content": docs_queried[0][0].page_content
        }
    ],
    max_tokens=4096,
    temperature=1.0,
    top_p=1.0,
    model=os.getenv("AZURE_OPENAI_CHAT_MODEL_DEPLOYMENT_NAME")
)

print(response.choices[0].message.content)

Under **Article 6** of the GDPR, processing of personal data is considered lawful only if at least one of the following **lawful bases** applies:

1. **Consent**  
   The data subject has freely, specifically, explicitly, and informedly given their consent to the processing of their personal data for one or more specified purposes.

2. **Contract**  
   Processing is necessary for the performance of a contract to which the data subject is a party, or for taking steps at the data subject's request prior to entering into a contract.

3. **Legal Obligation**  
   Processing is necessary for compliance with a legal obligation to which the data controller is subject (e.g., tax reporting or regulatory requirements).

4. **Vital Interests**  
   Processing is necessary to protect the vital interests of the data subject or another natural person. This typically applies in situations involving life-threatening emergencies.

5. **Public Interest or Official Authority**  
   Processing is necessa

In [ ]:
deployment = CHAT_DEPLOYMENT
response = chat_client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant that helps GDPR Compliance related topics.",
        },
        {
            "role": "user",
            "content": "What are the lawful bases for processing under GDPR Article 6?",
        }
    ],
    max_tokens=4096,
    temperature=1.0,
    top_p=1.0,
    model=deployment
)

print(response.choices[0].message.content)

In [ ]:
# ---------- Indexing ----------
def build_or_refresh_index(chunks: List[Document]) -> AzureSearch:
    vs = make_vectorstore()
    # add_documents will embed each chunk via embedding_function
    vs.add_documents(chunks)
    return vs

# Wrong query 

In [ ]:
docs_queried = vs.similarity_search_with_relevance_scores(
    query="What are the administrative fine tiers and assessment criteria in GDPR Article 83?",
    k=5,
)
print(docs_queried[0][0].page_content)

In [ ]:
deployment = CHAT_DEPLOYMENT
response = chat_client.chat.completions.create(
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant that helps GDPR Compliance related topics.",
        },
        {
            "role": "user",
            "content": "What are the lawful bases for processing under GDPR Article 6?",
        },
        {
            "role": "assistant", 
            "content": docs_queried[0][0].page_content
        }
    ],
    max_tokens=4096,
    temperature=1.0,
    top_p=1.0,
    model=deployment
)

from pprint import pprint
pprint(response)
pprint(response.choices[0].message.content)

In [ ]:
pprint(response.choices[0].message.content)

In [ ]:
# Retrieve Docs
query = "What are the lawful bases for processing under GDPR Article 6?"

docs_queried = vs.similarity_search_with_relevance_scores(
    query=query,
    k=3,  # Increased to 3 for better grounding
)

# Merge all retrieved chunks into one text block
retrieved_context = "\n\n---\n\n".join(
    [doc[0].page_content for doc in docs_queried]
)

deployment = CHAT_DEPLOYMENT

response = chat_client.chat.completions.create(
    model=deployment,
    max_tokens=1024,
    temperature=0.2,  # lower temp = less hallucination
    top_p=1.0,
    messages=[
        {
            "role": "system",
            "content": (
                "You are a GDPR Compliance Expert. Answer ONLY using the provided context.\n"
                "If the answer is not explicitly stated in the context, respond with:\n"
                "'The answer is not found in the provided GDPR dataset.'\n\n"
                "CRITICAL RULES:\n"
                "1. Use only facts from the context below (do NOT use external knowledge).\n"
                "2. Include citations by referencing Article or Recital numbers if present.\n"
                "3. Do NOT hallucinate, guess, or add extra info not present in the context.\n"
                "4. Be concise, accurate, and compliant.\n"
            ),
        },
        {
            "role": "user",
            "content": (
                f"Context:\n{retrieved_context}\n\n"
                f"Question: {query}\n\n"
                "Provide the answer with relevant citations."
            ),
        },
    ],
)

print(response.choices[0].message.content)


In [ ]:
# Retrieve Docs
query = "What are the lawful bases for processing under GDPR Article 1000, section 18, of IPC 800 and EU Act 12?"

docs_queried = vs.similarity_search_with_relevance_scores(
    query=query,
    k=3,  # Increased to 3 for better grounding
)

# Merge all retrieved chunks into one text block
retrieved_context = "\n\n---\n\n".join(
    [doc[0].page_content for doc in docs_queried]
)

deployment = CHAT_DEPLOYMENT

response = chat_client.chat.completions.create(
    model=deployment,
    max_tokens=1024,
    temperature=0.2,  # lower temp = less hallucination
    top_p=1.0,
    messages=[
        {
            "role": "system",
            "content": (
                "You are a GDPR Compliance Expert. Answer ONLY using the provided context.\n"
                "If the answer is not explicitly stated in the context, respond with:\n"
                "'The answer is not found in the provided GDPR dataset.'\n\n"
                "CRITICAL RULES:\n"
                "1. Use only facts from the context below (do NOT use external knowledge).\n"
                "2. Include citations by referencing Article or Recital numbers if present.\n"
                "3. Do NOT hallucinate, guess, or add extra info not present in the context.\n"
                "4. Be concise, accurate, and compliant.\n"
            ),
        },
        {
            "role": "user",
            "content": (
                f"Context:\n{retrieved_context}\n\n"
                f"Question: {query}\n\n"
                "Provide the answer with relevant citations."
            ),
        },
    ],
)

print(response.choices[0].message.content)
